In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
!pip install emoji
!pip install tensorflow
!pip install transformers

     |████████████████████████████████| 175 kB 4.2 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=c933600e2bff935ee87f4a7c68bf1a56046ffe11a44d4cd36df8d2b636635390
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji
     |████████████████████████████████| 462 kB 5.3 MB/s 
     |████████████████████████████████| 4.2 MB 4.6 MB/s 
     |████████████████████████████████| 6.6 MB 35.5 MB/s 
     |████████████████████████████████| 84 kB 2.0 MB/s 
     |████████████████████████████████| 596 kB 36.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
  
tokenizer = AutoTokenizer.from_pretrained('rabindralamsal/BERTsent')

model = TFAutoModelForSequenceClassification.from_pretrained('rabindralamsal/BERTsent')

Downloading:   0%|          | 0.00/323 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/824k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/929 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/515M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at rabindralamsal/BERTsent.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [3]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
link = 'https://drive.google.com/file/d/1Cus0W0LLYX7nhl6XLV50iINRh72cRCEv/view?usp=sharing'

In [5]:
# to get the id part of the file
id = link.split("/")[-2]
 
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('Survey_Results_Labeled_Feb.csv') 
 
df = pd.read_csv('Survey_Results_Labeled_Feb.csv')
df


,Unnamed: 0,text,1,2,3,Label
0,Q1,"""Human rights groups have raised alarm over wh...",0.0,0.7,0.3,neutral
1,Q2,@KingJames You have been awfully and painfully...,0.0,0.0,1.0,negative
2,Q3,"""@PrillLee @MarthaP29033882 @DanRather It is n...",0.0,0.3,0.7,negative
3,Q4,@AntoineSallesP @JoshuaHabib4 @JLMelenchon Why...,0.3,0.5,0.2,neutral
4,Q5,"@IAmMrKobayashi World War III; Full video, How...",0.2,0.7,0.1,neutral
...,...,...,...,...,...,...
97,Q96,There are 7 or 8 Billionaires in the UKRAINE! ...,0.4,0.0,0.6,negative
98,Q97,"""@EP_President @Europarl_EN @EU_Commission STO...",0.0,0.3,0.7,negative
99,Q98,"""@vonderleyen STOP RUSSIAN AGGRESSION AGAINST ...",0.0,0.1,0.9,negative
100,Q99,"""""However difficult it may be, our pain will b...",0.4,0.1,0.5,negative


In [6]:
df['text'] = df['text'].astype('str') 

In [7]:
docs = list(df.loc[:, "text"].values)

In [8]:
from tqdm import tqdm
scores = []

for i in tqdm(range(len(docs))):
    input = tokenizer.encode(docs[i], return_tensors="tf")
    output = model.predict(input)[0]
    prediction = tf.nn.softmax(output, axis=1).numpy()
    sentiment = np.argmax(prediction)
    
    scores.append({"BERTsent Label": sentiment,
                       "Negative": prediction[0,0],
                       "Neutral": prediction[0,1],
                       "Positive": prediction[0,2]
                  })

100%|██████████| 102/102 [01:02<00:00,  1.64it/s]


In [9]:
sentiments_score = pd.DataFrame.from_dict(scores)
df = df.join(sentiments_score)

In [10]:
df

,Unnamed: 0,text,1,2,3,Label,BERTsent Label,Negative,Neutral,Positive
0,Q1,"""Human rights groups have raised alarm over wh...",0.0,0.7,0.3,neutral,0,0.796714,0.199936,0.003350
1,Q2,@KingJames You have been awfully and painfully...,0.0,0.0,1.0,negative,0,0.837614,0.158905,0.003481
2,Q3,"""@PrillLee @MarthaP29033882 @DanRather It is n...",0.0,0.3,0.7,negative,0,0.765243,0.230464,0.004293
3,Q4,@AntoineSallesP @JoshuaHabib4 @JLMelenchon Why...,0.3,0.5,0.2,neutral,0,0.718155,0.277754,0.004092
4,Q5,"@IAmMrKobayashi World War III; Full video, How...",0.2,0.7,0.1,neutral,0,0.749426,0.246661,0.003913
...,...,...,...,...,...,...,...,...,...,...
97,Q96,There are 7 or 8 Billionaires in the UKRAINE! ...,0.4,0.0,0.6,negative,1,0.227966,0.717758,0.054276
98,Q97,"""@EP_President @Europarl_EN @EU_Commission STO...",0.0,0.3,0.7,negative,0,0.948297,0.047886,0.003817
99,Q98,"""@vonderleyen STOP RUSSIAN AGGRESSION AGAINST ...",0.0,0.1,0.9,negative,0,0.935105,0.060250,0.004644
100,Q99,"""""However difficult it may be, our pain will b...",0.4,0.1,0.5,negative,0,0.882522,0.108324,0.009154


In [11]:
#delete control tweets from set
df = df.drop(33)
df = df.drop(67)

In [12]:
#change column names to make clear what is what
df = df.rename(columns={"Label":"Manual Label"})

In [13]:
df

,Unnamed: 0,text,1,2,3,Manual Label,BERTsent Label,Negative,Neutral,Positive
0,Q1,"""Human rights groups have raised alarm over wh...",0.0,0.7,0.3,neutral,0,0.796714,0.199936,0.003350
1,Q2,@KingJames You have been awfully and painfully...,0.0,0.0,1.0,negative,0,0.837614,0.158905,0.003481
2,Q3,"""@PrillLee @MarthaP29033882 @DanRather It is n...",0.0,0.3,0.7,negative,0,0.765243,0.230464,0.004293
3,Q4,@AntoineSallesP @JoshuaHabib4 @JLMelenchon Why...,0.3,0.5,0.2,neutral,0,0.718155,0.277754,0.004092
4,Q5,"@IAmMrKobayashi World War III; Full video, How...",0.2,0.7,0.1,neutral,0,0.749426,0.246661,0.003913
...,...,...,...,...,...,...,...,...,...,...
97,Q96,There are 7 or 8 Billionaires in the UKRAINE! ...,0.4,0.0,0.6,negative,1,0.227966,0.717758,0.054276
98,Q97,"""@EP_President @Europarl_EN @EU_Commission STO...",0.0,0.3,0.7,negative,0,0.948297,0.047886,0.003817
99,Q98,"""@vonderleyen STOP RUSSIAN AGGRESSION AGAINST ...",0.0,0.1,0.9,negative,0,0.935105,0.060250,0.004644
100,Q99,"""""However difficult it may be, our pain will b...",0.4,0.1,0.5,negative,0,0.882522,0.108324,0.009154


In [14]:
df['BERTsent Label'].replace({0: "negative", 1: "neutral", 2: 'positive'}, inplace=True)

In [17]:
 import sklearn
 from sklearn.metrics import balanced_accuracy_score, precision_recall_fscore_support
 b_a_Vader = balanced_accuracy_score(df['Manual Label'], df['BERTsent Label'])
 p_r_f1_class_Vader = precision_recall_fscore_support(df['Manual Label'], df['BERTsent Label'], average=None, labels=['negative', 'neutral', 'positive'])
 p_r_f1_Vader = precision_recall_fscore_support(df['Manual Label'], df['BERTsent Label'], average='macro')

In [18]:
print(b_a_Vader)
print(p_r_f1_Vader)
print(p_r_f1_class_Vader)

0.5940711462450593
(0.7953453453453454, 0.5940711462450593, 0.6256995586136048, None)
(array([0.68333333, 0.7027027 , 1.        ]), array([0.89130435, 0.59090909, 0.3       ]), array([0.77358491, 0.64197531, 0.46153846]), array([46, 44, 10]))


In [ ]:
import pandas as pd
y_actu = df['Manual Label']
y_pred = df['BERTsent Label']
df_confusion = pd.crosstab(y_actu, y_pred)

In [ ]:
df_confusion

BERTsent Label,negative,neutral,positive
Manual Label,,,
negative,41,5,0
neutral,18,26,0
positive,1,6,3


In [ ]:
#Get accuracies per class order (negative, neutral, positive)
from sklearn.metrics import confusion_matrix
y_true = df['Manual Label']
y_pred = df['BERTsent Label']
matrix = confusion_matrix(y_true, y_pred)
matrix.diagonal()/matrix.sum(axis=1)

array([0.89130435, 0.59090909, 0.3       ])

In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
df.to_csv('BERTsent_Test_Feb.csv')
!cp BERTsent_Test_Feb.csv "/content/drive/My Drive/Colab Notebooks"

